**1. Python environment setup for data analysis and visualization.**

In [ ]:
import altair as alt
import pandas as pd
import os
import numpy as np

**2. Checks if a folder named "TNL" exists, and if not, it creates one and prints a confirmation message. If it already exists, it will print message saying it exists.**

In [ ]:
import os

# Specify the folder name
folder_name = "TNL"

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")


Folder 'TNL' created successfully.


**3. Download a zip file from GitHUb using the requests library in Python. It gets the file, locally saves it with the same name then prints a confirmation message. Ensure you have necessary permissions to download the file and potential exceptions that may happen during download process are handled.**

In [ ]:
import requests

url = 'https://github.com/stefanbund/grus-m2/raw/main/lob_caps%202.zip' # grab the archive in github
local_filename = url.split('/')[-1]

response = requests.get(url)            # download it
with open(local_filename, 'wb') as f:
    f.write(response.content)

print(f"Zip file downloaded as '{local_filename}'.")    # prove it's downloaded


Zip file downloaded as 'lob_caps%202.zip'.


**4. Code above will extract contents of the zip file by zip_file_path into the target_folder.**

In [ ]:
import zipfile

# Specify the path to the downloaded zip file
zip_file_path = local_filename

# Specify the target folder where you want to extract the contents
target_folder = folder_name

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

print(f"Contents extracted to '{target_folder}'.")


Contents extracted to 'TNL'.


### Assemble a global data set from 300+ daily csv files

*lob_caps* is the location for more than 365 days of day-long data collection, polled every 15 seconds from coinbase.

**5. Creating a function for processing CSV files.  It defines a function to retrieve filenames that contain a specific type from a directory, reads each CSV file into a DataFrame, concatenates them into a single DataFrame, and then sorts it by time.**

In [ ]:
folder_loc = "/content/TNL/lob_caps" #/content/lob_caps/Fri Apr 07 2023 12:26:51 GMT-0700 (Pacific Daylight Time)-MEANSHIFT.csv

def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)  ./lob_caps/
    ret = []  #list
    for root, dirs, files in os.walk(folder_loc): #core/gh-code/grus-code/ver2-pctChangeDriven/lob_caps
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = folder_loc +"/"+ filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  303127
start:  1660221600292.0  end:  1693780654910.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


### The Basic Features

These features were assembled from another sampling mechanism that delivers an aggregated digest of the limit order book, which is complex and will contain upwards of 20,000 data points, each sample.

**6. Display the first two rows of the capsFrame DataFrame by using .head(2) method in pandas.This will output the first two entries in the DataFrame. If this code is ran in this Python environment, we’ll be able to see the data contained in the first two rows of capsFrame.**

In [ ]:
capsFrame.head(2)

,bc,ac,tbv,tav,time,mp,minBid
0,4278178.73,34127182.98,460583.04,320612.61,1.676736e+12,19.74,19.66
1,4275769.45,34126881.89,460460.79,320597.36,1.676736e+12,19.74,19.66


### Impute Missing Values

**7. Using the fillna method with the method='ffill' parameter will impute the missing values with the last non-null value in each column. This is a common technique known as "forward filling".**


In [ ]:
# impute missing values with last non-null value
capsFrame['bc'] = capsFrame['bc'].fillna(method='ffill')
capsFrame['ac'] = capsFrame['ac'].fillna(method='ffill')
capsFrame['tbv'] = capsFrame['tbv'].fillna(method='ffill')
capsFrame['tav'] = capsFrame['tav'].fillna(method='ffill')
capsFrame['mp'] = capsFrame['mp'].fillna(method='ffill')
capsFrame['minBid'] = capsFrame['minBid'].fillna(method='ffill')

In [ ]:
capsFrame.to_csv("global_data.csv")

### Discover Precursor and Surge Episodes

The goal of the data prep is to discover periods of continuous, positive momentum. These are **surges**.

The periods preceding surges are, for the sake of the experiment, **precursors**. They are detected as periods of discontinuous positive momentum, or negative momentum.

A ten-row window is used to calculate positive or negative momentum. A percent **change** is calculated for the ten row subsample.

### Regularization of Critical Features

Get percent change as basis for comprehending LOB.

Create new columns which depict the momentum of one row versus the next, in terms of price , capitalization and volume.

**8. Create a copy of the original DataFrame to preserve the data. The pct_change() function with the periods parameter set to the lookback period will calculate the percentage change for each specified column.**

In [ ]:
# Load the time series data into a pandas dataframe
# consider changing this approach because it doesn't actually check in between values

caps_df = capsFrame # make a copy of the original, just in case we ruin it
lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
caps_df['bc_change'] = caps_df['bc'].pct_change(periods=lookback_period)
caps_df['ac_change'] = caps_df['ac'].pct_change(periods=lookback_period)
caps_df['tav_change'] = caps_df['tav'].pct_change(periods=lookback_period)
caps_df['tbv_change'] = caps_df['tbv'].pct_change(periods=lookback_period)
## key components: bc_change, ac_change, tav_change, tbv_change, change
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of the asset over a fixed lookback period

303127 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change', 'bc_change',
       'ac_change', 'tav_change', 'tbv_change'],
      dtype='object')


### Establish benchmarks for percent change

The mean of change represents the average rate of change between LOB samples. This is used to determine whether the change between rows is significant or not

**9. To calculate the mean change metric for the 'change' column in the caps_df DataFrame, use the .mean() method in pandas. Display the mean change with rounding to 8 decimal places bu using round() function with 8 as the second parameter after .mean()**


In [ ]:
# for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

0.00054026

###Data Mining: Sequence Discovery

Define precursors from surges, prepare the data with this sequence:

- Precursor --> Surge

- Prepare to cluster every precursor, by the sequential, resultant surge. Do not assume causality, but rather preoccurance.

- Use the threshold, mean change as tool to separate precursor from surges, where surges represent periods of positive momentum above threshold.

- This step defines the data schema for the remainder of the process, where key statistics are defined for precursors and surges.

**10. Code for identifying periods where the percentage change is greater or less than the mean change threshold. It separates the data into two categories: surges, where there is a spike in price, and precursors, where the price is flat or negative.This will create two lists of dictionaries, surges and precursors, each containing relevant information about the time and magnitude of price changes. Additionally, handle cases where the DataFrame might not be divisible by 10, as the last window may have fewer than 10 rows.**

In [ ]:
# identify units of 10 rows where the percent change is greater or less than the threshold
### key components: bc_change, ac_change, tav_change, tbv_change, change
threshold = meanChange
surges = [] # list of moments when there is a spike in the price
precursors = [] # list of moments when the price is not spking, is flat or negative
for i in range(0,len(caps_df),10):
    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                            'type':'precursor',
                            'precursor_buy_cap_pct_change':caps_df.iloc[i]['bc_change'],
                            'precursor_ask_cap_pct_change':caps_df.iloc[i]['ac_change'],
                            'precursor_bid_vol_pct_change':caps_df.iloc[i]['tbv_change'],
                            'precursor_ask_vol_pct_change':caps_df.iloc[i]['tav_change']
                            })

Confirm you have a database of precursors and surges

**11. This ouputs the  details of the first two ‘surge’ events captured in the analysis. If code is ran in this Python environment, it will display the dictionary values for each item in the console.**

In [ ]:
for item in surges[:2]:
  print(item)

{'time': 1676736134155.0, 's_MP': 19.73, 'change': 0.0006084678658852205, 'type': 'surge'}
{'time': 1676736357582.0, 's_MP': 19.75, 'change': 0.0008103830813230184, 'type': 'surge'}


**12. This outputs the details  of the first two ‘precursor’ events captured in the analysis. If code is ran in this Python environment, it will display the dictionary values for each item in the console.**

In [ ]:
for item in precursors[:2]:
  print(item)

{'time': 1676735918245.0, 'p_MP': 19.74, 'change': nan, 'type': 'precursor', 'precursor_buy_cap_pct_change': nan, 'precursor_ask_cap_pct_change': nan, 'precursor_bid_vol_pct_change': nan, 'precursor_ask_vol_pct_change': nan}
{'time': 1676735947783.0, 'p_MP': 19.75, 'change': 0.0001519756838906039, 'type': 'precursor', 'precursor_buy_cap_pct_change': -0.0033524826579651723, 'precursor_ask_cap_pct_change': 0.0003187180731083483, 'precursor_bid_vol_pct_change': -0.0015807572940592651, 'precursor_ask_vol_pct_change': 0.001717274938125568}


###Pre-process: Merge precursors and surges into time series

A dataframe of sequences, **sequence_df** is created by concatenating both buckets, and sorting by time. This will create a time series of surge and precursor periods, as defined by:

- 10 windows percent change values

- Contiguity: these precursor and surges are next to each other and thus have a length or duration of momentum.

**13. This code will give a combined DataFrame that includes both surges and precursors, sorted by the time they occured.**

In [ ]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True)

**14. The .index attribute in pandas returns the index or the row labels of the DataFrame. This will display the index of sequence_df, which will be useful for understanding how the data is organized, after sorting the DataFrame.**

In [ ]:
sequence_df.index

Int64Index([ 1557,  2726,  2727,  2728,  2729,  2730,  2731,  2732,  1558,
             1559,
            ...
            14685,  7592,  7593, 14686,  7594, 14687,  7595, 14688, 14689,
             7596],
           dtype='int64', length=30313)

###View the aligned, continuous time series of precursors and surges

View the final abstraction: sets of precursor periods, next to surges, in a linear time series. Precursors effectively precede surges on a linear time series.

**15. Display the first 40 entries of the ‘type’ column from the sequence_df DataFrame, by using the .head() method in pandas with the number 40 passed as an argument. This will output the ‘type’ information for the first 40 rows in the DataFrame. If code is ran in this Python environment, it will show the data contained in the ‘type’ column for those rows.**

In [ ]:
sequence_df['type'].head(40)

1557        surge
2726    precursor
2727    precursor
2728    precursor
2729    precursor
2730    precursor
2731    precursor
2732    precursor
1558        surge
1559        surge
2733    precursor
1560        surge
2734    precursor
2735    precursor
2736    precursor
1561        surge
2737    precursor
2738    precursor
1562        surge
2739    precursor
2740    precursor
2741    precursor
2742    precursor
2743    precursor
2744    precursor
2745    precursor
1563        surge
1564        surge
2746    precursor
2747    precursor
1565        surge
1566        surge
2748    precursor
1567        surge
1568        surge
2749    precursor
2750    precursor
2751    precursor
2752    precursor
1569        surge
Name: type, dtype: object

###Visualize proof of algorithmic accuracy

This chart will plot the price time series, with an area of precursor and surge, as proof of our algorithmic accuracy.

**16. Altair visualization code is set up to create an interactive chart that combines line and bar marks to represent the data mining accuracy, distinguishing between surges and precursors in the dataset. The chart is titled “Data Mining Accuracy, Surge vs Precursor Sequence” with a subtitle explaining the precursors.**

  **Here’s a breakdown of the code:**
-   **subset takes the first 4999 rows of sequence_df.**
-   **line creates a green line chart for the ‘s_MP’ column over time.**
-   **s_bar creates a bar chart for the ‘s_MP’ column over time, colored by ‘type’.**
-   **p_bar creates a bar chart for the ‘p_MP’ column over time, also colored by ‘type’.**
-   **chart combines these three charts and sets the width and height.**
-   **The title and subtitle are set for the chart, and it’s made interactive.**


In [ ]:
subset = sequence_df[:4999]
line = alt.Chart(subset).mark_line(color='green').encode(
    x='time',
    y='s_MP'
)

s_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='s_MP',
    color='type:N'
)

p_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='p_MP',
    color='type:N'
)

chart = (s_bar + p_bar + line).properties(width=600, height=500)
chart.title = 'Data Mining Accuracy, Surge vs Precursor Sequence'
subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
chart.interactive()

alt.LayerChart(...)

**17. Display the column names of the sequence_df DataFrame, by using the .columns attribute. This will output the names of all the columns in sequence_df. If this code is ran in the Python environment, it will show the the column names of sequence_df.**


In [ ]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change'],
      dtype='object')

###Data Mining 2: Information gain, feature engineering

- Perform information gain on grouped precursors and surges.

- Define the **sum change** or total change per continuous episode (precursor or surge).

- Define the **length** of each episode.

- Define the height of the surge, how high did the continuous positive momentum reach?

- Define the size (area) of the sruge, as a triangular area (height times length), as **surge_area**

- Create one line to describe a precursor or surge and its related order book statistics

**18. This code is designed to create a new column called ‘group’ in the sequence_df DataFrame, which groups rows based on changes in the ‘type’ column. Then, for selected columns related to percentage changes, it applies a transformation within each group. The transformation sums the values of each column unless all values are NaN, in which case it assigns NaN.**

In [ ]:
sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
columns_to_transform = [
    'precursor_buy_cap_pct_change',
    'precursor_ask_cap_pct_change',
    'precursor_bid_vol_pct_change',
    'precursor_ask_vol_pct_change'
]

for col in columns_to_transform:
      sequence_df[col] = sequence_df.groupby('group')[col].transform(lambda x: x.sum() if not x.isna().all() else np.nan)

**19. Here’s a breakdown of what the code does:**

**1. sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()**
  - This line creates a new column ‘group’ that increments every time there is a change in the ‘type’ column. It’s a clever way to group consecutive rows with the same ‘type’.

**2. columns_to_transform = [...]**

  - You define a list of column names that you want to transform.

**3. for col in columns_to_transform: ...**

  - You iterate over each column in the list and apply a transformation to the groups defined by the ‘group’ column.

**4. sequence_df[col] = sequence_df.groupby('group')[col].transform(...)**

  - For each group, you sum the values of the column unless all values are NaN, in which case you keep it as NaN.

**This approach is useful for aggregating data within groups while handling missing data appropriately.**


In [ ]:
# # impute missing values with last non-null value DONE PRIOR, NOW AT START
sequence_df['s_MP'] = sequence_df['s_MP'].fillna(method='ffill')
sequence_df['p_MP'] = sequence_df['p_MP'].fillna(method='ffill')
sequence_df['precursor_buy_cap_pct_change'] = sequence_df['precursor_buy_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_cap_pct_change'] = sequence_df['precursor_ask_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_bid_vol_pct_change'] = sequence_df['precursor_bid_vol_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_vol_pct_change'] = sequence_df['precursor_ask_vol_pct_change'].fillna(method='ffill')

###Critical Grouped Features

- In this phase of feature engineering, the statistics that summarize each sequence are aggregated by group.
- We engineer 10 new features, to help describe the success of the surge, including three features that help describe how the max price in a  precursor is reached by the price action in the subsequent spike.

**20. This code performs several group-based calculations on the sequence_df DataFrame. Here’s a summary of what each line does:**



**1. sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')**
  - This adds a ‘length’ column to sequence_df, representing the count of rows in each (‘type’, ‘group’) combination.

**2. print(sequence_df.shape[0])**
  - This prints the total number of rows in sequence_df.

**3. sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')**
  - This adds a ‘sum_change’ column to sequence_df, representing the sum of ‘change’ values within each group.

**4. sequence_df['max_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('max')**
  - This adds a ‘max_surge_mp’ column to sequence_df, representing the maximum ‘s_MP’ value within each surge group.

**5. sequence_df['min_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('min')**
  - This adds a ‘min_surge_mp’ column to sequence_df, representing the minimum ‘s_MP’ value within each surge group.
  
**6. sequence_df['max_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('max')**
  - This adds a ‘max_precursor_mp’ column to sequence_df, representing the maximum ‘p_MP’ value within each precursor group.

**7. sequence_df['min_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('min')**
  - This adds a ‘min_precursor_mp’ column to sequence_df, representing the minimum ‘p_MP’ value within each precursor group.

**8. sequence_df['area'] = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)**
  - This calculates an ‘area’ column as the product of ‘length’ and ‘sum_change’ for each row.

**9. sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']**
  - This creates a ‘surge_area’ column, which is equal to ‘area’ for surge types.

**10. sequence_df['surge_targets_met_pct'] = sequence_df.apply(lambda group: ((group['max_precursor_mp']-group['max_surge_mp'])/group['max_surge_mp'] ) *100, axis=1)**
  - This calculates the percentage difference between ‘max_precursor_mp’ and ‘max_surge_mp’ for each group.





In [ ]:
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')

print(sequence_df.shape[0])
sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')

sequence_df['max_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('max')
sequence_df['min_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('min')

sequence_df['max_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('max')
sequence_df['min_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('min')

sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)

sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']

sequence_df['surge_targets_met_pct']  = sequence_df.apply(lambda group: ((group['max_precursor_mp']-group['max_surge_mp'])/group['max_surge_mp']  ) *100, axis=1)

30313


**21. Prove that the new groupby operations generated the columns needed. Use .columns attribute. .shape[0] returns the total amount of records in DataFrame.**

In [ ]:
print(sequence_df.columns)
print(sequence_df.shape[0])

Index(['time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change', 'group',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_area',
       'surge_targets_met_pct'],
      dtype='object')
30313


**22. Display the first five records by using .head(5) method.**

In [ ]:
sequence_df.head(5)

,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,group,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
1557,1.660222e+12,29.96,0.551053,surge,NaN,NaN,NaN,NaN,NaN,1,1,0.551053,29.96,29.96,NaN,NaN,0.551053,0.551053,NaN
2726,1.660222e+12,29.96,-0.000367,precursor,29.97,-0.001248,0.000049,-0.000355,0.006605,2,7,-0.004474,29.96,29.96,29.99,29.85,-0.031318,NaN,0.100134
2727,1.660222e+12,29.96,-0.000500,precursor,29.99,-0.001248,0.000049,-0.000355,0.006605,2,7,-0.004474,29.96,29.96,29.99,29.85,-0.031318,NaN,0.100134
2728,1.660222e+12,29.96,-0.001201,precursor,29.93,-0.001248,0.000049,-0.000355,0.006605,2,7,-0.004474,29.96,29.96,29.99,29.85,-0.031318,NaN,0.100134
2729,1.660222e+12,29.96,-0.001370,precursor,29.89,-0.001248,0.000049,-0.000355,0.006605,2,7,-0.004474,29.96,29.96,29.99,29.85,-0.031318,NaN,0.100134


###Data Mining 3: Form Final Sequences By Statistical Weight

Group the sequences by their type, then order them in sequential order.

**23. This will create a new DataFrame called unique_df by grouping sequence_df by the ‘group’ column, taking the first row from each group, and then resetting the index. This is a common operation when removing duplicate entries based on a grouping variable and keep only the first occurrence of each group.**

  **Here's what each part of the code does:**
  

*   **sequence_df.groupby('group'): This groups the DataFrame by the ‘group’ column.**
*   **.first(): This takes the first row from each group.**
*   **.reset_index(): This resets the index of the resulting DataFrame.**

  **After running this code, unique_df will contain unique groups from sequence_df with all the columns present in the original DataFrame.**

In [ ]:
unique_df = sequence_df.groupby('group').first().reset_index()

Display three lines to check the result.

**24. Display the first three records by using the .head() method with 3 as an argument.**

In [ ]:
unique_df.head(3)

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,1,1.660222e+12,29.96,0.551053,surge,NaN,NaN,NaN,NaN,NaN,1,0.551053,29.96,29.96,NaN,NaN,0.551053,0.551053,NaN
1,2,1.660222e+12,29.96,-0.000367,precursor,29.97,-0.001248,0.000049,-0.000355,0.006605,7,-0.004474,29.96,29.96,29.99,29.85,-0.031318,NaN,0.100134
2,3,1.660222e+12,29.89,0.000603,surge,29.85,-0.001248,0.000049,-0.000355,0.006605,2,0.001407,29.89,29.86,29.85,29.85,0.002815,0.002815,-0.133824


Clean it up: remove the first surge, which is almost always empty, because it lacks data.

Create two dataframes: those with even and odd index positions, then merge them into a potential final dataframe.

**25. By removing the first surge, we ensure that the sequence starts with a precursor, which aligns with the analysis requirements. Then, by separating the DataFrame into even and odd rows, we effectively create two new DataFrames: one for precursors and one for surges. Finally, concatenating these two DataFrames side by side allow us to analyze the relationship between each precursor and the subsequent surge.**

  **Here's a summary of the code:**


*  **unique_df = unique_df.iloc[1:]: This line removes the first row from unique_df, which is assumed to be a surge, to ensure the sequence starts with a precursor.**

*   **even_df = unique_df.iloc[::2].reset_index(drop=True): This creates a DataFrame of precursors by selecting every second row starting from the first row (which is now a precursor).**


*   **odd_df = unique_df.iloc[1::2].reset_index(drop=True): This creates a DataFrame of surges by selecting every second row starting from the second row.**

*   **merged_df = pd.concat([even_df, odd_df], axis=1): This merges the two DataFrames side by side, pairing each precursor with the following surge.**

  **The resulting merged_df will have each precursor and its corresponding surge in the same row, allowing for a paired analysis.**


In [ ]:
# needs to start with a precursor removes the first surge
unique_df = unique_df.iloc[1:]
even_df = unique_df.iloc[::2].reset_index(drop=True) # precursors
odd_df = unique_df.iloc[1::2].reset_index(drop=True) # surges

merged_df = pd.concat([even_df, odd_df], axis=1) # combine into one row, with one precursor and one surge, A=>B

Merged dataframe represents a precursor married to a surge. This suggests that there was a precursor, then a subsequent spike, this A=>B event is captured on each line.

**26. Output the first 10 entries in the DataFrame, showing the relationship between each precursor and its corresponding surge. If this code is ran in this Python environment, it will show the data for these pairs.**

In [ ]:
merged_df[:10] # represents a precursor married to a spike/surge

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.96,-0.000367,precursor,29.97,-0.001248,0.000049,-0.000355,0.006605,...,0.006605,2,0.001407,29.89,29.86,29.85,29.85,0.002815,0.002815,-0.133824
1,4,1.660222e+12,29.86,0.000435,precursor,29.90,-0.003274,0.000034,-0.001139,0.004397,...,0.004397,1,0.000602,29.95,29.95,29.90,29.90,0.000602,0.000602,-0.166945
2,6,1.660222e+12,29.95,-0.001370,precursor,29.91,-0.001283,-0.000030,-0.000091,-0.003342,...,-0.003342,1,0.002315,29.89,29.89,29.80,29.80,0.002315,0.002315,-0.301104
3,8,1.660222e+12,29.89,-0.000335,precursor,29.87,0.000776,0.000022,0.000155,0.002564,...,0.002564,1,0.001306,29.89,29.89,29.85,29.85,0.001306,0.001306,-0.133824
4,10,1.660223e+12,29.89,0.000201,precursor,29.95,0.007328,-0.000040,0.004069,-0.000544,...,-0.000544,2,0.004537,29.65,29.52,29.52,29.52,0.009075,0.009075,-0.438449
5,12,1.660225e+12,29.65,0.000236,precursor,29.64,0.002045,-0.000001,0.000652,-0.000564,...,-0.000564,2,0.004600,29.69,29.56,29.60,29.60,0.009199,0.009199,-0.303132
6,14,1.660226e+12,29.69,0.000303,precursor,29.71,-0.018628,-0.000003,-0.006599,-0.000756,...,-0.000756,2,0.004272,29.76,29.75,29.71,29.71,0.008545,0.008545,-0.168011
7,16,1.660228e+12,29.75,-0.005863,precursor,29.68,-0.015495,-0.000265,-0.002403,-0.022496,...,-0.022496,1,0.001489,29.28,29.28,29.13,29.13,0.001489,0.001489,-0.512295
8,18,1.660230e+12,29.28,-0.006492,precursor,29.12,0.007772,-0.000080,0.003752,-0.004560,...,-0.004560,1,0.001966,29.09,29.09,29.04,29.04,0.001966,0.001966,-0.171880
9,20,1.660232e+12,29.09,-0.005528,precursor,29.06,-0.012941,0.000030,-0.003469,0.006498,...,0.006498,2,0.002670,28.83,28.83,28.85,28.85,0.005339,0.005339,0.069372


Drop null valued items, along the rows (axis 1)

**27. Remove columns that only contain null values from the merged_df DataFrame and display the first few rows of the resulting final_df. This will create a new DataFrame final_df without the columns that have all null values and then print the first five rows. If this code is ran in this Python environment, it will show you the cleaned DataFrame.**


In [ ]:
final_df = merged_df.dropna(axis=1, how='all') # purge null rows
final_df.head()

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.96,-0.000367,precursor,29.97,-0.001248,0.000049,-0.000355,0.006605,...,0.006605,2,0.001407,29.89,29.86,29.85,29.85,0.002815,0.002815,-0.133824
1,4,1.660222e+12,29.86,0.000435,precursor,29.90,-0.003274,0.000034,-0.001139,0.004397,...,0.004397,1,0.000602,29.95,29.95,29.90,29.90,0.000602,0.000602,-0.166945
2,6,1.660222e+12,29.95,-0.001370,precursor,29.91,-0.001283,-0.000030,-0.000091,-0.003342,...,-0.003342,1,0.002315,29.89,29.89,29.80,29.80,0.002315,0.002315,-0.301104
3,8,1.660222e+12,29.89,-0.000335,precursor,29.87,0.000776,0.000022,0.000155,0.002564,...,0.002564,1,0.001306,29.89,29.89,29.85,29.85,0.001306,0.001306,-0.133824
4,10,1.660223e+12,29.89,0.000201,precursor,29.95,0.007328,-0.000040,0.004069,-0.000544,...,-0.000544,2,0.004537,29.65,29.52,29.52,29.52,0.009075,0.009075,-0.438449


Talk about the features we engineered during the experiment. We now possess a dataframe with significant width beyond our initial 5 dimensions (bc, ac, tbv, tav, mp).

**28. Displaying the columns of final_df by using the .columns attribute**

In [ ]:
final_df.columns

Index(['group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_targets_met_pct',
       'group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_area',
       'surge_targets_met_pct'],
      dtype='object')

Draft a set of columns that include the targets met, but eliminate the duplicate columns - this is for surges only.

**29. Renaming the last column of the final_df DataFrame to ‘surge_targets_met_pct.1’. This line of code will take all but the last column name from final_df, add the new column name ‘surge_targets_met_pct.1’, and then assign this list back to the DataFrame’s columns. Now, the last column of final_df will be renamed as specified.**


In [ ]:
final_df.columns = list(final_df.columns[:-1]) + ['surge_targets_met_pct.1']

#Binning Process

Establish/define a series of bins that express how much prices in the precursor surges, to the target of 1% or more.

**30. Create a set of bins for categorizing the ‘surge_targets_met_pct’ values in the final_df DataFrame. The bins are designed to divide the range of values into specific intervals, and labels are created for each bin. Here’s the breakdown of the bins and their corresponding labels: When code is ran, it will print out the range of each bin with two decimal places. This is useful for understanding how the ‘surge_targets_met_pct’ values are distributed across different ranges.**

In [ ]:
bins = [
    final_df['surge_targets_met_pct'].min() -1,  # Min value  # -4 to 0 divided into three equal parts
    -4/3,# Second bin edge for negative values
    -4/6,
    -4/12,
    0,
    0.125,
    0.25, 0.5, 0.75, 1,  # Four bins between 0 and 1
    2,  # One bin between 1 and 2
    final_df['surge_targets_met_pct'].max()+ 1]
bin_labels = list(range(1, len(bins)))

for i in range(1, len(bins)):
    print(f'Bin {bin_labels[i-1]}: {bins[i-1]:.2f} - {bins[i]:.2f}')

Bin 1: -8.04 - -1.33
Bin 2: -1.33 - -0.67
Bin 3: -0.67 - -0.33
Bin 4: -0.33 - 0.00
Bin 5: 0.00 - 0.12
Bin 6: 0.12 - 0.25
Bin 7: 0.25 - 0.50
Bin 8: 0.50 - 0.75
Bin 9: 0.75 - 1.00
Bin 10: 1.00 - 2.00
Bin 11: 2.00 - 5.28


Create a new feature called 'label' that helps distinguish the category of bin, for the precursor's sruge.

By now, we can describe the types of precursors we wish to trade, by label.

**31. We’ve successfully used the pd.cut function to categorize the ‘surge_targets_met_pct’ values into discrete bins. The new ‘label’ column in final_df will contain the bin labels corresponding to the range in which each ‘surge_targets_met_pct’ value falls.. This line creates a new column called ‘label’ in final_df that assigns a label to each ‘surge_targets_met_pct’ value based on the bins you defined earlier. The pd.cut function is a great way to segment and sort data values into bins.
If this code is ran, each entry in the ‘surge_targets_met_pct’ column will be assigned a categorical label that indicates which bin it belongs to, making it easier to analyze the distribution of the data.**


In [ ]:
final_df['label'] = pd.cut(final_df['surge_targets_met_pct'], bins=bins, labels=bin_labels)


#Write to CSV: Step One, Pipeline

Write the classified types to a csv file. We weill ingest this csv file in our machine learning phase, to help discover a predictive model.

Binned Pipeline: contains the labels.

**32. This command will save the final_df DataFrame to a CSV file named ‘binned_pipeline.csv’ without including the index. Here’s how the command works in a code block. This will create a CSV file in the current working directory. If this code is ran in this Python environment, the file ‘binned_pipeline.csv’ will be saved with the data from final_df.**


In [ ]:
final_df.to_csv('binned_pipeline.csv', index=False)

Binned Binary Pipeline: contains either a 1, or a 0, based on the attractiveness of the surge.

**33. We’ve created a new DataFrame final_df_binary by excluding the last column from final_df. Then, you’ve added a ‘label’ column to final_df_binary, where surges are labeled with a 1 if ‘surge_targets_met_pct’ is greater than 0.74, and 0 otherwise. Finally, you’re saving this DataFrame to a CSV file named ‘binary_binned_pipeline.csv’.. This will save the final_df_binary DataFrame to a CSV file in the current working directory. If this code is ran in this Python environment, ‘binary_binned_pipeline.csv’ will be created with the binary-labeled data.**

In [ ]:
final_df_binary = final_df.iloc[:, :-1]
final_df_binary['label'] = (final_df_binary['surge_targets_met_pct']> 0.74).astype(int) # label surges with either 0 or 1
final_df_binary.to_csv('binary_binned_pipeline.csv')